# SimpleITK Notebook Teasers

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk
from myshow import myshow, myshow3d

from downloaddata import fetch_data as fdata

## 10 Lines of Code for Segmentation

In [ ]:
# Load Data
img = sitk.ReadImage(fdata("cthead1.png"))

# Create a function with arguments we wish to experiment on and explore.
def ws_callback(img, sigma, level, markWatershedLine, fullyConnected):
    feature_img = sitk.GradientMagnitudeRecursiveGaussian(img, sigma=sigma)
    ws_img = sitk.MorphologicalWatershed(feature_img, level, markWatershedLine, fullyConnected)
    myshow(sitk.LabelToRGB(ws_img), "Watershed Segmentation")

# Connect parameters to interactive widgets
interact( lambda **kwargs: ws_callback(img, **kwargs),
             sigma=FloatSlider(min=0.05, max=5.0, step=0.05, value=1.0),
             markWatershedLine=True,
             fullyConnected=False,
             level=FloatSlider(min=0, max=255, step=0.1, value=4.0) )

Next we will demonstate a minimal example for image registration. We start by loading and visualizing two image.

In [ ]:
fixed_image =  sitk.ReadImage(fdata("RIRE/training_001_ct.mha"), sitk.sitkFloat32)
moving_image = sitk.ReadImage(fdata("RIRE/training_001_mr_T1.mha"), sitk.sitkFloat32) 

moving_resampled_pre = sitk.Resample(moving_image, fixed_image, sitk.Euler3DTransform(), sitk.sitkLinear, 0.0, moving_image.GetPixelIDValue())
myshow3d(0.5*fixed_image+0.5*moving_resampled_pre)

## 12 lines of Code for Multi-modal Registration

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100)
registration_method.SetOptimizerScalesFromPhysicalShift() 
registration_method.SetInitialTransform(initial_transform, inPlace=False)
final_transform = registration_method.Execute(fixed_image, moving_image)

In [ ]:
moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelIDValue())

myshow3d(0.5*fixed_image+0.5*moving_resampled)